# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/home/travis/miniconda/envs/test/lib/python3.7/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.20.1.60/12589/1 Dashboard: http://10.20.1.60:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,1035,George,0.6669978027746819,0.19070816090224585
2000-01-01 00:00:01,943,Victor,0.5755268543549221,0.8309546035622744
2000-01-01 00:00:02,1043,Charlie,-0.7758510654652373,-0.6276213259323093
2000-01-01 00:00:03,1020,Norbert,-0.39807703651946547,0.09029195267864476
2000-01-01 00:00:04,1011,Laura,0.8572904904202645,0.3518186269438184
2000-01-01 00:00:05,1045,Ingrid,0.5254278155128147,0.2780565038101137
2000-01-01 00:00:06,1026,Quinn,0.6438446549164334,-0.6680105360236201
2000-01-01 00:00:07,965,Bob,0.005706167232564008,-0.31009697753750665
2000-01-01 00:00:08,977,Alice,0.8781757708579283,-0.06077320275922293


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,1078,Tim,0.7383108287759488,-0.3251183724924134
2000-01-30 00:00:01,1009,Xavier,0.4461341657777673,0.3605313443882443
2000-01-30 00:00:02,968,Ursula,0.4393030229969346,-0.8064658012545804
2000-01-30 00:00:03,956,Jerry,0.6987586557650611,0.6016673422235179
2000-01-30 00:00:04,1018,Kevin,0.6032013896477633,-0.411691821277544
2000-01-30 00:00:05,958,Kevin,-0.9769208513679739,0.5559302543270355
2000-01-30 00:00:06,977,Frank,-0.24206770201229766,0.022040232512664515
2000-01-30 00:00:07,1012,Frank,0.6377293756193714,-0.08675881114959227
2000-01-30 00:00:08,973,Kevin,0.17100430510447406,0.43779020639386745


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1035,George,0.666998,0.190708
1,2000-01-01 00:00:01,943,Victor,0.575527,0.830955
2,2000-01-01 00:00:02,1043,Charlie,-0.775851,-0.627621
3,2000-01-01 00:00:03,1020,Norbert,-0.398077,0.090292
4,2000-01-01 00:00:04,1011,Laura,0.857290,0.351819


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1035,George,0.666998,0.190708
1,2000-01-01 00:00:01,943,Victor,0.575527,0.830955
2,2000-01-01 00:00:02,1043,Charlie,-0.775851,-0.627621
3,2000-01-01 00:00:03,1020,Norbert,-0.398077,0.090292
4,2000-01-01 00:00:04,1011,Laura,0.857290,0.351819


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 4.57 s, sys: 583 ms, total: 5.16 s
Wall time: 3.34 s


name
Alice       0.000690
Bob        -0.001392
Charlie    -0.002311
Dan        -0.000718
Edith       0.003101
Frank      -0.001567
George      0.000636
Hannah      0.001721
Ingrid     -0.001920
Jerry       0.000690
Kevin      -0.001649
Laura      -0.000146
Michael     0.000166
Norbert    -0.001112
Oliver      0.000360
Patricia   -0.001952
Quinn       0.000350
Ray         0.004656
Sarah       0.001709
Tim         0.001715
Ursula      0.003768
Victor     -0.000073
Wendy      -0.000367
Xavier      0.002635
Yvonne      0.001817
Zelda       0.003695
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.15.parquet  part.22.parquet  part.2.parquet
_metadata	  part.16.parquet  part.23.parquet  part.3.parquet
part.0.parquet	  part.17.parquet  part.24.parquet  part.4.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.5.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.6.parquet
part.12.parquet   part.1.parquet   part.27.parquet  part.7.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.8.parquet
part.14.parquet   part.21.parquet  part.29.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.42 s, sys: 138 ms, total: 1.56 s
Wall time: 1.05 s


name
Alice       0.000690
Bob        -0.001392
Charlie    -0.002311
Dan        -0.000718
Edith       0.003101
Frank      -0.001567
George      0.000636
Hannah      0.001721
Ingrid     -0.001920
Jerry       0.000690
Kevin      -0.001649
Laura      -0.000146
Michael     0.000166
Norbert    -0.001112
Oliver      0.000360
Patricia   -0.001952
Quinn       0.000350
Ray         0.004656
Sarah       0.001709
Tim         0.001715
Ursula      0.003768
Victor     -0.000073
Wendy      -0.000367
Xavier      0.002635
Yvonne      0.001817
Zelda       0.003695
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.23 s, sys: 113 ms, total: 1.34 s
Wall time: 941 ms


name
Alice       0.000690
Bob        -0.001392
Charlie    -0.002311
Dan        -0.000718
Edith       0.003101
Frank      -0.001567
George      0.000636
Hannah      0.001721
Ingrid     -0.001920
Jerry       0.000690
Kevin      -0.001649
Laura      -0.000146
Michael     0.000166
Norbert    -0.001112
Oliver      0.000360
Patricia   -0.001952
Quinn       0.000350
Ray         0.004656
Sarah       0.001709
Tim         0.001715
Ursula      0.003768
Victor     -0.000073
Wendy      -0.000367
Xavier      0.002635
Yvonne      0.001817
Zelda       0.003695
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html